In [28]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import torchvision
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time
import numpy as np

In [29]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 42
#torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

model_parameters_file = 'data/model_param.pth'

## Configure loaders

In [30]:
data_dir = 'data/'

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(
      data_dir,
      train=True,
      download=True,
      transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize((0.1307,), (0.3081,))
                             ])),
      batch_size=batch_size_train,
      shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(data_dir, train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                                 ])),
  batch_size=batch_size_test, shuffle=True)
    
type(test_loader)

torch.utils.data.dataloader.DataLoader

## Create network model

In [31]:
class ImageClass(nn.Module):
    def __init__(self):
        super(ImageClass, self).__init__()
        self.linear1 = nn.Linear(1 * 28 * 28, 100)
        self.linear2 = nn.Linear(100, 10)
        #self.linear3 = nn.Linear(20, 10)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        #x = F.relu(self.linear3(x))
        return F.log_softmax(x, dim=1)

In [32]:
#Network model
model = ImageClass()

#check if gpu is available
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
model.to(device)

#Stochastic gradient decent
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

In [33]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [34]:
def train(epoch):

  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    #move tensors to device
    data = data.to(device)
    target = target.to(device)
    #set parameters gradients to 0
    optimizer.zero_grad()
    
    #runs model with data and returns result in output
    #here data contains a 64 * 1 * 28 * 28 tensor, 64 is batch size
    output = model(data.reshape(-1, 1 * 28 * 28))
    
    #Use negative log likelihood loss.
    loss = F.nll_loss(output, target)
    
    #with this gradients are calculated
    loss.backward()
    
    #update gradients
    optimizer.step()
    
    #Display iteration statistics
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [35]:
def test():
  #Sets the module in evaluation mode
  model.eval()
  test_loss = 0
  correct = 0
    
  #dont update dynamic computation graph
  with torch.no_grad():
        #for every example in test
        for data, target in test_loader:
            #move tensors to device
            data = data.to(device)
            target = target.to(device)
            #evaluate the model
            output = model(data.reshape(-1, 1 * 28 * 28).to(device))
            #acumulate the loss
            test_loss += F.nll_loss(output, target, size_average=False).item()
            
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

## Run training for 3 epoch and then save the traning status

In [36]:
training_checkpoint_file = 'data/training_checkpoint.pth'
#Check accuaracy before training
test()
tic = time.perf_counter()

for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()
    
toc = time.perf_counter()

print(f"Time elapsed: {toc - tic:0.4f} seconds")

training_checkpoint = {
    "epoch": epoch,
    "model_state": model.state_dict(),
    "optim_state": optimizer.state_dict() 
}

torch.save(training_checkpoint, training_checkpoint_file)


Test set: Avg. loss: 2.3173, Accuracy: 1070/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.324821
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.204351
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.933007
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.747426
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.534667
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.293316
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.286038
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.192320
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.035127
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.016887
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.771619
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.856270
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.960124
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.839487
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.569525
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.004233
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.753264
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.850529
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.092844
Train Epoch: 1 [12160

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.726847
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.431699
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.370808
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.542975
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.499721
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.789955
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.701373
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.399677
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.435858
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.554836
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.307100
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.613855
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.539800
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.486687
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.463843
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.505607
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.429957
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.537487
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.448300
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.428262


## Using saved training checkpoint, train some aditional epoch

In [42]:
training_checkpoint_file = 'data/training_checkpoint.pth'

loaded_checkpoint = torch.load(training_checkpoint_file)

additional_epoch = 2

#Set past parameters to start training again
current_epoch = loaded_checkpoint['epoch']
#rewrite model and optimizer objects with previous saved state
model = ImageClass()
model.load_state_dict(loaded_checkpoint['model_state'])
optimizer = optim.SGD(model.parameters(), lr=0)
optimizer.load_state_dict(loaded_checkpoint['optim_state'])

#Check accuaracy before training

test()
tic = time.perf_counter()

for epoch in range(current_epoch + 1, current_epoch + additional_epoch + 1):
  train(epoch)
  test()
    
toc = time.perf_counter()

print(f"Time elapsed: {toc - tic:0.4f} seconds")

training_checkpoint = {
    "epoch": epoch,
    "model_state": model.state_dict(),
    "optim_state": optimizer.state_dict() 
}

torch.save(training_checkpoint, training_checkpoint_file)


Test set: Avg. loss: 0.3927, Accuracy: 8504/10000 (85%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.443193
Train Epoch: 5 [640/60000 (1%)]	Loss: 0.689084
Train Epoch: 5 [1280/60000 (2%)]	Loss: 0.264930
Train Epoch: 5 [1920/60000 (3%)]	Loss: 0.333603
Train Epoch: 5 [2560/60000 (4%)]	Loss: 0.445818
Train Epoch: 5 [3200/60000 (5%)]	Loss: 0.304277
Train Epoch: 5 [3840/60000 (6%)]	Loss: 0.370792
Train Epoch: 5 [4480/60000 (7%)]	Loss: 0.503324
Train Epoch: 5 [5120/60000 (9%)]	Loss: 0.411509
Train Epoch: 5 [5760/60000 (10%)]	Loss: 0.474785
Train Epoch: 5 [6400/60000 (11%)]	Loss: 0.365316
Train Epoch: 5 [7040/60000 (12%)]	Loss: 0.451881
Train Epoch: 5 [7680/60000 (13%)]	Loss: 0.415654
Train Epoch: 5 [8320/60000 (14%)]	Loss: 0.317033
Train Epoch: 5 [8960/60000 (15%)]	Loss: 0.238958
Train Epoch: 5 [9600/60000 (16%)]	Loss: 0.328833
Train Epoch: 5 [10240/60000 (17%)]	Loss: 0.347376
Train Epoch: 5 [10880/60000 (18%)]	Loss: 0.591328
Train Epoch: 5 [11520/60000 (19%)]	Loss: 0.429911
Train Epoch: 5 [12160

Train Epoch: 6 [44160/60000 (74%)]	Loss: 0.407470
Train Epoch: 6 [44800/60000 (75%)]	Loss: 0.285602
Train Epoch: 6 [45440/60000 (76%)]	Loss: 0.444945
Train Epoch: 6 [46080/60000 (77%)]	Loss: 0.510984
Train Epoch: 6 [46720/60000 (78%)]	Loss: 0.496203
Train Epoch: 6 [47360/60000 (79%)]	Loss: 0.349778
Train Epoch: 6 [48000/60000 (80%)]	Loss: 0.251950
Train Epoch: 6 [48640/60000 (81%)]	Loss: 0.373215
Train Epoch: 6 [49280/60000 (82%)]	Loss: 0.530927
Train Epoch: 6 [49920/60000 (83%)]	Loss: 0.328673
Train Epoch: 6 [50560/60000 (84%)]	Loss: 0.245422
Train Epoch: 6 [51200/60000 (85%)]	Loss: 0.312018
Train Epoch: 6 [51840/60000 (86%)]	Loss: 0.275771
Train Epoch: 6 [52480/60000 (87%)]	Loss: 0.503903
Train Epoch: 6 [53120/60000 (88%)]	Loss: 0.486866
Train Epoch: 6 [53760/60000 (90%)]	Loss: 0.508628
Train Epoch: 6 [54400/60000 (91%)]	Loss: 0.162930
Train Epoch: 6 [55040/60000 (92%)]	Loss: 0.542614
Train Epoch: 6 [55680/60000 (93%)]	Loss: 0.413017
Train Epoch: 6 [56320/60000 (94%)]	Loss: 0.261742


## Save model parameters

In [43]:

torch.save(model.state_dict(), model_parameters_file)

## Load and test model using only saved parameters

In [44]:
model_for_inference = ImageClass()

model_for_inference.load_state_dict(torch.load(model_parameters_file))
#set model for evaluation
model_for_inference.eval()

test_loss = 0
correct = 0
    
#dont update dynamic computation graph
with torch.no_grad():
    #for every example in test
    for data, target in test_loader:
        #move tensors to device
        data = data.to(device)
        target = target.to(device)
        #evaluate the model
        output = model_for_inference(data.reshape(-1, 1 * 28 * 28).to(device))
        #acumulate the loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
            
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
        
test_loss /= len(test_loader.dataset)
test_losses.append(test_loss)

print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))


Test set: Avg. loss: 0.3765, Accuracy: 8553/10000 (86%)



## Todo

NA

## Notes

- Based on: https://nextjournal.com/gkoehler/pytorch-mnist